Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 3 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)

Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

Логичность и чистота кода (1 балл)

In [95]:
import re
import nltk
import requests
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup

nltk.download("punkt")
morph = MorphAnalyzer()

[nltk_data] Downloading package punkt to /home/marynepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Пройдемся по страницам с отзывами в кинопоиске, скачаем тексты отзывов, деля их на положительные и отрицательные.

In [86]:
revs = {'pos':[], 'neg': []}
response = requests.get('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/bad/perpage/50/', headers={'User-agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(response,'html.parser')
for l in soup.find_all('span', {'class':'_reachbanner_', 'itemprop':'reviewBody'})[:50]:
    revs['neg'].append(l.get_text())

In [88]:
response = requests.get('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/good/perpage/50/', headers={'User-agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(response,'html.parser')
for l in soup.find_all('span', {'class':'_reachbanner_', 'itemprop':'reviewBody'})[:50]:
    revs['pos'].append(l.get_text())

Отделим тестовую выборку

In [101]:
X_test = revs['pos'][40:] + revs['neg'][40:]
Y_test = ['pos']*10 + ['neg']*10

In [94]:
print(revs)

{'pos': ["Единственная мысль, которая возникает во время просмотра данного фильма, исключая красивый и привлекательный антураж, выдержанные сцены в специфическом для данной картины колорите, это тема романтической любви, коими прототипами являются Ромео и Джульета (книга данного произведения даже появляется во 2 части и упоминается на уроке литературы), а также различные произведения эпохи рыцарских походов аля Дон Кихот, в которых тема любви раскрывается через пожертвование чем-то ради другого, близкого тебе человека, вплоть до самой смерти, тоже самое мы можем лицезреть и в данном фильме, герои буквально влюбляются в друг друга с одного взгляда, их чувства в самом начале настолько скованны, что кажется, как будто актеры плохо сыграли, но, как мне видится, в этом и была задумка режиссерской команды. В конечном итоге, как и во всех мелодрамах, герои не могут оставаться друг без друга, стоит им лишь разлучиться, как их сразу же кидает в морально-духовные переживания переходящие в депрес

Токенизируем и лемматизируем отзывы и составляем словарь со словами, которые встречаются либо только в положительных, либо только в отрицательных отзывах.

In [141]:
def words_to_voc(text):
    
    tokens = nltk.word_tokenize(text.lower())
    words = []
    for t in tokens:
        if str(morph.parse(',')[0].tag) is 'PNCT':
            words.append(morph.parse(t)[0].normal_form)
            
    return words
            
def make_voc(texts):
    
    pos_words = Counter()
    neg_words = Counter()
    
    for t in texts['pos'][:40]:
        pos_words.update(words_to_voc(t))
    for t in texts['neg'][:40]:
        neg_words.update(words_to_voc(t))
        
    pos_w = set(pos_words.keys()) - set(neg_words.keys())
    neg_w = set(neg_words.keys()) - set(pos_words.keys())
    
    words = {'pos':[], 'neg': []}
    
    for w in pos_w:
        if pos_words[w] > 2:
            words['pos'].append(w)
            
    for w in neg_w:
        if neg_words[w] > 2:
            words['neg'].append(w)
    
    return words

voc = make_voc(revs)

Создадим функцию, которая предсказывает, положительный ли отзыв или отрицательный.

In [144]:
def mark_text(text):
    words = words_to_voc(text) #можно еще сделать сет из этого, чтобы не учитывать повторение, но у меня так accuracy хуже
    n = 0
    p = 0
    for w in words:
        if w in voc['pos']:
            p += 1
        elif w in voc['neg']:
            n += 1
    if p >= n:
        return 'pos'
    else:
        return 'neg'

И проверим accuracy.

In [145]:
def check_acc(x, y):
    res = []  
    for text in x:
        res.append(mark_text(text))
    print("Accuracy: %.4f" % accuracy_score(res, y))
    
check_acc(X_test, Y_test)

Accuracy: 0.8500


Улучшение:

1) Учитывать частотность слов и добавить веса в соответствии с ними

2) Посмотреть на n-граммы